## 1. Import necessary libraries and load data 

In [2]:
import pandas as pd
from sklearn.externals import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
import xml.etree.ElementTree as ET
from lxml import etree
from scipy.sparse import hstack
import numpy as np
import warnings
import spacy
from os.path import join

#nlp = spacy.load('en_core_web_sm')

path_train = "data/semeval_16/ABSA16_Laptops_Train_English_SB2.xml"
path_test = "data/semeval_16/EN_LAPT_SB2_TESTB.xml"

#For stanford POS Tagger
home = "stanford-postagger-full-2018-10-16"
from nltk.tag.stanford import StanfordPOSTagger as POS_Tag
from nltk import word_tokenize

_path_to_model = home + "/models/english-bidirectional-distsim.tagger"
_path_to_jar = home + "/stanford-postagger.jar"
stanford_tag = POS_Tag(model_filename=_path_to_model, path_to_jar=_path_to_jar)

/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/nltk/tag/stanford.py:149: DeprecationWarning: 
The StanfordTokenizer will be deprecated in version 3.2.5.
Please use nltk.tag.corenlp.CoreNLPPOSTagger or nltk.tag.corenlp.CoreNLPNERTagger instead.
  super(StanfordPOSTagger, self).__init__(*args, **kwargs)


## 2. Preprocessing 

In [3]:
#xml parser
def get_list(path):
    tree=ET.parse(path)
    root = tree.getroot()
    text_list = []
    opinion_list = []
    for review in root.findall('Review'):
        text_string=""
        opinion_inner_list=[]
        for sent in review.findall('./sentences/sentence'):
            text_string= text_string+ " "+ sent.find('text').text
        text_list.append(text_string)
        for opinion in review.findall('./Opinions/Opinion'):
            opinion_dict = {
                opinion.get('category').replace('#','_'): opinion.get('polarity')
            }
            opinion_inner_list.append(opinion_dict)
        opinion_list.append(opinion_inner_list)
    return text_list,opinion_list

In [4]:
#Selecting only 20 most common aspect.
def get_most_common_aspect(opinion_list):
    import nltk
    opinion= []
    for inner_list in opinion_list:
        for _dict in inner_list:
            for key in _dict:
                opinion.append(key)
    most_common_aspect = [k for k,v in nltk.FreqDist(opinion).most_common(20)]
    return most_common_aspect

In [5]:
#generate data frame
def get_data_frame(text_list,opinion_list,most_common_aspect):
    data={'Review':text_list}
    df = pd.DataFrame(data)
    if opinion_list:
        for inner_list in opinion_list:
            for _dict in inner_list:
                for key in _dict:
                    if key in most_common_aspect:
                        df.loc[opinion_list.index(inner_list),key]=_dict[key]
    return df

In [6]:
#generate data frame for aspect extraction task
def get_aspect_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['positive','negative','neutral','conflict'],[1,1,1,1])
    df = df.fillna(0)
    return df

In [7]:
def get_positive_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['positive'],[1])
        df[common_aspect]=df[common_aspect].replace(['negative','neutral','conflict'],[0,0,0])
    df = df.fillna(0)
    return df

In [8]:
def get_negative_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['negative'],[1])
        df[common_aspect]=df[common_aspect].replace(['positive','neutral','conflict'],[0,0,0])
    df = df.fillna(0)
    return df

In [9]:
def get_neutral_data_frame(df,most_common_aspect):
    for common_aspect in most_common_aspect:
        df[common_aspect]=df[common_aspect].replace(['neutral','conflict'],[1,1])
        df[common_aspect]=df[common_aspect].replace(['negative','positive'],[0,0])
    df = df.fillna(0)
    return df

In [10]:
#To tag using stanford pos tagger
def posTag(review):
    tagged_text_list=[]
    for text in review:
        tagged_text_list.append(stanford_tag.tag(word_tokenize(text)))
    return tagged_text_list
#posTag("this is random text")

In [11]:
#Filter the word with tag- noun,adjective,verb,adverb
def filterTag(tagged_review):
    final_text_list=[]
    for text_list in tagged_review:
        final_text=[]
        for word,tag in text_list:
            if tag in ['NN','NNS','NNP','NNPS','RB','RBR','RBS','JJ','JJR','JJS','VB','VBD','VBG','VBN','VBP','VBZ']:
                final_text.append(word)
        final_text_list.append(' '.join(final_text))
    return final_text_list

In [12]:
def get_dict_aspect(y,most_common_aspect):
    position=[]
    for innerlist in y:
        position.append([i for i, j in enumerate(innerlist) if j == 1])
    sorted_common=sorted(most_common_aspect)
    dict_aspect=[]
    for innerlist in position:
        inner_dict={}
        for word in sorted_common:
            if sorted_common.index(word) in innerlist:
                inner_dict[word]= 5
            else:
                inner_dict[word]=0
        dict_aspect.append(inner_dict)
    return dict_aspect

# Stage 1 : Opinion prediction

### 2.1 PosTag train and test files and serialize them 

In [13]:
#Stage 1:
#Making list to train
train_text_list,train_opinion_list = get_list(path_train)
most_common_aspect = get_most_common_aspect(train_opinion_list)

In [14]:
#This takes time to tag. Already tagged and saved. So, loading file ...
#tagged_text_list_train=posTag(train_text_list)
#joblib.dump(tagged_text_list_train, 'tagged_text_list_train.pkl')
tagged_text_list_train=joblib.load('d/tagged_text_list_train.pkl')

In [15]:
#train list after filter
final_train_text_list=filterTag(tagged_text_list_train)

In [16]:
#get data frame
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_train_aspect = get_aspect_data_frame(df_train,most_common_aspect)
df_train_aspect = df_train_aspect.reindex_axis(sorted(df_train_aspect.columns), axis=1)

/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.


In [23]:
#df_train_aspect

In [18]:
#Similar for test list
test_text_list,test_opinion_list = get_list(path_test)

In [19]:
#tagged_text_list_test=posTag(test_text_list)
#joblib.dump(tagged_text_list_test, 'tagged_text_list_test.pkl')
tagged_text_list_test=joblib.load('d/tagged_text_list_test.pkl')

In [86]:
#tagged_text_list_test

In [20]:
final_test_text_list=filterTag(tagged_text_list_test)

In [38]:
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)
df_test_aspect = get_aspect_data_frame(df_test,most_common_aspect)
df_test_aspect = df_test_aspect.reindex_axis(sorted(df_test_aspect.columns), axis=1)

/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [87]:
#df_test

In [88]:
#df_test_aspect

In [89]:
df_test_aspect.replace('',1, inplace=True)
#df_test_aspect

### 2.2 Get necessary dataframes and mumpy arrays for text and labels respectively

In [45]:
#Sort the data frame according to aspect's name and separate data(X) and target(y)
#df_train_aspect = df_train_aspect.sample(frac=1).reset_index(drop=True) #For randoming
X_train= df_train_aspect.Review
y_train = df_train_aspect.drop('Review',1)

#df_test_aspect = df_test_aspect.sample(frac=1).reset_index(drop=True) #For randoming
X_test = df_test_aspect.Review
y_test = df_test_aspect.drop('Review',1)

In [36]:
#y_train

In [96]:
#y_test

In [46]:
#Change y_train to numpy array
import numpy as np
y_train = np.asarray(y_train, dtype=np.int64)
y_test = np.asarray(y_test, dtype=np.int64)

### 2.3 Generate word embeddings with sklearn's countvectorizer

In [ ]:
#Generate word vecotors using CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
vect = CountVectorizer(max_df=1.0,stop_words='english')  
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

## 3. Training - I. Naive Bayes', II. SVC, III. Linear SVC, IV. SGD

In [97]:
#import xgboost as xgb

In [109]:
#X_train_aspect_dtm
#y_train

In [98]:
#Create various models. These are multi-label models.
nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
C = 1.0 #SVregularization parameter
svc = OneVsRestClassifier(svm.SVC(kernel='linear', C=C)).fit(X_train_dtm, y_train)
lin_svc = OneVsRestClassifier(svm.LinearSVC(C=C)).fit(X_train_dtm, y_train)
sgd = OneVsRestClassifier(SGDClassifier()).fit(X_train_dtm,y_train)
#xg_model = xgb.XGBClassifier().fit(X_train_dtm, y_train)

In [49]:
#Predict the test data using classifiers
y_pred_class = nb_classif.predict(X_test_dtm)
y_pred_class_svc = svc.predict(X_test_dtm)
y_pred_class_lin_svc = lin_svc.predict(X_test_dtm)
y_pred_class_sgd = sgd.predict(X_test_dtm)

In [50]:
#Following code to test metrics of all aspect extraction classifiers
from sklearn import metrics

## 4. Evaluation

In [51]:
print(metrics.accuracy_score(y_test,y_pred_class))
print(metrics.accuracy_score(y_test,y_pred_class_svc))
print(metrics.accuracy_score(y_test,y_pred_class_lin_svc))
print(metrics.accuracy_score(y_test,y_pred_class_sgd))

0.025
0.05
0.05
0.0125


In [52]:
print(metrics.precision_score(y_test,y_pred_class,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.precision_score(y_test,y_pred_class_sgd,average='micro'))

0.75
0.7112299465240641
0.7321937321937322
0.696875


In [53]:
print(metrics.recall_score(y_test,y_pred_class,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.recall_score(y_test,y_pred_class_sgd,average='micro'))

0.4576271186440678
0.6440677966101694
0.6222760290556901
0.5399515738498789


In [54]:
print(metrics.f1_score(y_test,y_pred_class,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_svc,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_lin_svc,average='micro'))
print(metrics.f1_score(y_test,y_pred_class_sgd,average='micro'))

0.5684210526315788
0.6759847522236341
0.6727748691099477
0.6084583901773534


In [55]:
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print(metrics.classification_report(y_test, y_pred_class))
    print(metrics.classification_report(y_test, y_pred_class_svc))
    print(metrics.classification_report(y_test, y_pred_class_lin_svc))
    print(metrics.classification_report(y_test, y_pred_class_sgd))

              precision    recall  f1-score   support

           0       0.67      0.14      0.24        14
           1       0.71      0.50      0.59        24
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00        21
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         7
           7       0.76      0.64      0.69        39
           8       1.00      1.00      1.00        80
           9       0.44      0.17      0.24        24
          10       0.62      0.70      0.65        46
          11       0.00      0.00      0.00         5
          12       0.57      0.30      0.39        27
          13       0.57      0.45      0.50        29
          14       0.77      0.33      0.47        30
          15       0.00      0.00      0.00         4
          16       0.00      0.00      0.00         9
          17       0.00    

## Stage 2 : Aspect category prediction

In [56]:
#Stage 2:
#Generating extra feature that indicates which aspect category is present in the review
train_dict_aspect=get_dict_aspect(y_train, most_common_aspect)
d_train=DictVectorizer() 
X_train_aspect_dtm = d_train.fit_transform(train_dict_aspect)

#y_test is used to generated extra feature in order to test the performance of 2nd classifer.
#Use y_pred_class_svc(Highest performer for aspect classification) as input for extra feature to test the overall performace.
test_dict_aspect=get_dict_aspect(y_test,most_common_aspect)
d_test=DictVectorizer() 
X_test_aspect_dtm = d_test.fit_transform(test_dict_aspect)

In [69]:
#Function for classiflying positive,negative or neutral sentiment of all the aspects
def classify_sentiment(df_train,df_test,X_train_aspect_dtm,X_test_aspect_dtm):
    
    df_train = df_train.reindex_axis(sorted(df_train_positive.columns), axis=1)
    df_test = df_test.reindex_axis(sorted(df_test_positive.columns), axis=1)
    
    df_test.replace('',1, inplace=True)
    #print(df_test.iloc[0])

    import numpy as np
    X_train = df_train.Review
    y_train = df_train.drop('Review',1)
    y_train = np.asarray(y_train, dtype=np.int64)

    X_test = df_test.Review
    y_test = df_test.drop('Review',1)
    y_test = np.asarray(y_test, dtype=np.int64)

    vect_sen = CountVectorizer(stop_words='english',ngram_range=(1,2))  
    X_train_dtm = vect_sen.fit_transform(X_train)
    X_test_dtm = vect_sen.transform(X_test)

    #combining word vector with extra feature.
    from scipy.sparse import hstack
    X_train_dtm=hstack((X_train_dtm, X_train_aspect_dtm))
    X_test_dtm=hstack((X_test_dtm, X_test_aspect_dtm))

    C = 1.0 #SVregularization parameter
    nb_classif = OneVsRestClassifier(MultinomialNB()).fit(X_train_dtm, y_train)
    svc = OneVsRestClassifier(svm.SVC(kernel='linear', C=C)).fit(X_train_dtm, y_train)
    lin_svc = OneVsRestClassifier(svm.LinearSVC(C=C)).fit(X_train_dtm, y_train)
    sgd = OneVsRestClassifier(SGDClassifier()).fit(X_train_dtm,y_train)

    y_pred_class= nb_classif.predict(X_test_dtm)
    y_pred_class_svc = svc.predict(X_test_dtm)
    y_pred_class_lin_svc = lin_svc.predict(X_test_dtm)
    y_pred_class_sgd = sgd.predict(X_test_dtm)
    return (y_test,y_pred_class,y_pred_class_svc,y_pred_class_lin_svc,y_pred_class_sgd)

In [70]:
def print_metrices(y_test,y_pred_class,y_pred_class_svc,y_pred_class_lin_svc,y_pred_class_sgd):
    print("Accuracy:")
    print(metrics.accuracy_score(y_test,y_pred_class))
    print(metrics.accuracy_score(y_test,y_pred_class_svc))
    print(metrics.accuracy_score(y_test,y_pred_class_lin_svc))
    print(metrics.accuracy_score(y_test,y_pred_class_sgd))

    print("\nAverage precision:")
    print(metrics.precision_score(y_test,y_pred_class,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.precision_score(y_test,y_pred_class_sgd,average='micro'))

    print("\nAverage recall:")
    print(metrics.recall_score(y_test,y_pred_class,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.recall_score(y_test,y_pred_class_sgd,average='micro'))
    
    print("\nAverage f1:")
    print(metrics.f1_score(y_test,y_pred_class,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_svc,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_lin_svc,average='micro'))
    print(metrics.f1_score(y_test,y_pred_class_sgd,average='micro'))

    print("\nClassification report:")
    print(metrics.classification_report(y_test, y_pred_class))
    print(metrics.classification_report(y_test, y_pred_class_svc))
    print(metrics.classification_report(y_test, y_pred_class_lin_svc))
    print(metrics.classification_report(y_test, y_pred_class_sgd))

In [71]:
#For positive sentiment classifier
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_positive = get_positive_data_frame(df_train,most_common_aspect)
df_test_positive = get_positive_data_frame(df_test,most_common_aspect)
y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos=classify_sentiment(df_train_positive,df_test_positive,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos)

/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """


Accuracy:
0.025
0.325
0.3
0.225

Average precision:
1.0
0.976271186440678
0.9959349593495935
1.0

Average recall:
0.2033898305084746
0.6973365617433414
0.5932203389830508
0.5835351089588378

Average f1:
0.33802816901408456
0.8135593220338984
0.7435508345978756
0.7370030581039755

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.00      0.00      0.00        24
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00        21
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00        39
           8       1.00      0.88      0.93        80
           9       0.00      0.00      0.00        24
          10       1.00      0.30      0.47        46
          11       0.00      0.00      0.00    

In [72]:
#For negative sentiment classifier
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_neg = get_negative_data_frame(df_train,most_common_aspect)
df_test_neg = get_negative_data_frame(df_test,most_common_aspect)

y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg=classify_sentiment(df_train_neg,df_test_neg,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg)

/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """


Accuracy:
0.0
0.075
0.05
0.0875

Average precision:
1.0
0.9861111111111112
1.0
0.9824561403508771

Average recall:
0.024213075060532687
0.17191283292978207
0.13075060532687652
0.13559322033898305

Average f1:
0.04728132387706856
0.29278350515463913
0.23126338329764454
0.23829787234042552

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.00      0.00      0.00        24
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00        21
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00        39
           8       1.00      0.11      0.20        80
           9       0.00      0.00      0.00        24
          10       0.00      0.00      0.00        46
          11       0.00      0.00     

In [73]:
#For neutral or conflict sentiment classifier
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)
df_test = get_data_frame(final_test_text_list,test_opinion_list,most_common_aspect)

df_train_neu = get_neutral_data_frame(df_train,most_common_aspect)
df_test_neu = get_neutral_data_frame(df_test,most_common_aspect)

y_test_neu,y_pred_class_neu,y_pred_class_svc_neu,y_pred_class_lin_svc_neu,y_pred_class_sgd_neu=classify_sentiment(df_train_neu,df_test_neu,X_train_aspect_dtm,X_test_aspect_dtm)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print_metrices(y_test_neu,y_pred_class_neu,y_pred_class_svc_neu,y_pred_class_lin_svc_neu,y_pred_class_sgd_neu)

/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """


Accuracy:
0.0
0.0
0.0
0.0

Average precision:
0.0
0.9230769230769231
1.0
1.0

Average recall:
0.0
0.029055690072639227
0.007263922518159807
0.007263922518159807

Average f1:
0.0
0.05633802816901409
0.014423076923076922
0.014423076923076922

Classification report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        14
           1       0.00      0.00      0.00        24
           2       0.00      0.00      0.00        12
           3       0.00      0.00      0.00         4
           4       0.00      0.00      0.00        21
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         7
           7       0.00      0.00      0.00        39
           8       0.00      0.00      0.00        80
           9       0.00      0.00      0.00        24
          10       0.00      0.00      0.00        46
          11       0.00      0.00      0.00         5
          12       0.00      0.00

# Full pipeline : ABSA of user input

In [74]:
#Aspect Based Sentiment analyis of user's input.
user_input=input("Enter a laptop review:\n\n")
#Enter a laptop review:

#This is my first asus laptop. So far i am really enjoying this laptop. 512GB SSD is super fast. 
#Battery life is also good and can last very long. 
#I have no complain on screen quality too as display supports 4k videos. Maybe that is why it costs a lot. 
#This is an expensive laptop and it's price is very high compared to other laptops of similar specs. 
#So, if you have no trouble paying for this laptop, it is pretty good.

#Preprocessing and vectorizing
tagged_user_input = posTag([user_input])
filter_tagged_user_input = filterTag(tagged_user_input)

user_input_series=pd.Series(filter_tagged_user_input)
user_input_series_dtm=vect.transform(user_input_series)

predict_aspect= svc.predict(user_input_series_dtm)
extra_feature=get_dict_aspect(predict_aspect, most_common_aspect)
extra_feature_dtm=DictVectorizer().fit_transform(extra_feature)
predict_aspect

Enter a laptop review:

This is my first asus laptop. So far i am really enjoying this laptop. 512GB SSD is super fast. Battery life is also good and can last very long. I have no complain on screen quality too as display supports 4k videos. Maybe that is why it costs a lot. This is an expensive laptop and it's price is very high compared to other laptops of similar specs. So, if you have no trouble paying for this laptop, it is pretty good.


array([[1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]])

In [75]:
#predicting weather the dectected aspect is positive or not
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_positive = get_positive_data_frame(df_train,most_common_aspect)
y_test_pos,y_pred_class_pos,y_pred_class_svc_pos,y_pred_class_lin_svc_pos,y_pred_class_sgd_pos=classify_sentiment(df_train_positive,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_pos

/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """


array([[1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [76]:
#predicting weather the dectected aspect is negative or not
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_negative = get_negative_data_frame(df_train,most_common_aspect)
y_test_neg,y_pred_class_neg,y_pred_class_svc_neg,y_pred_class_lin_svc_neg,y_pred_class_sgd_neg=classify_sentiment(df_train_negative,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_neg

/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [77]:
#predicting weather the dectected aspect is neutral or coflict or not
test_opinion_list=[]
df_test = get_data_frame(filter_tagged_user_input,test_opinion_list,most_common_aspect)
df_train = get_data_frame(final_train_text_list,train_opinion_list,most_common_aspect)

df_train_neutral = get_neutral_data_frame(df_train,most_common_aspect)
y_test_neu,y_pred_class_neu,y_pred_class_svc_neu,y_pred_class_lin_svc_neu,y_pred_class_sgd_neu=classify_sentiment(df_train_neutral,df_test,X_train_aspect_dtm,extra_feature_dtm)

y_pred_class_svc_neu

/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  after removing the cwd from sys.path.
/anaconda3/envs/nlp_vir/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """


array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

In [78]:
#Finding the aspect that is positive
index_positive=[]
for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_pos.tolist()[0])):
    if a ==1 and b==1:
        index_positive.append(i)
index_positive         

[0, 3, 8, 10, 13]

In [79]:
#Finding the aspect that is negative
index_negative=[]
for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_neg.tolist()[0])):
    if a ==1 and b==1:
        index_negative.append(i)
index_negative         

[]

In [80]:
#Finding the aspect that is neutral
index_neutral=[]
for i, (a, b) in enumerate(zip(predict_aspect.tolist()[0], y_pred_class_svc_neu.tolist()[0])):
    if a ==1 and b==1:
        index_neutral.append(i)
index_neutral         

[12]

In [81]:
output=[]

In [82]:
if index_positive:
    for index in index_positive:
        output.append(sorted(most_common_aspect)[index]+": positive")

In [83]:
if index_negative:
    for index in index_negative:
        output.append(sorted(most_common_aspect)[index]+": negative")

In [84]:
if index_neutral:
    for index in index_neutral:
        output.append(sorted(most_common_aspect)[index]+": neutral or conflict")

In [85]:
#Prediction of Aspect Based Sentiment Analaysis for user's input
output

['BATTERY_OPERATION_PERFORMANCE: positive',
 'DISPLAY_GENERAL: positive',
 'LAPTOP_GENERAL: positive',
 'LAPTOP_OPERATION_PERFORMANCE: positive',
 'LAPTOP_QUALITY: positive',
 'LAPTOP_PRICE: neutral or conflict']